In [2]:
!pip install -qU langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 19.2 MB/s eta 0:00:00


In [3]:
with open('/content/email_text.eml', 'r') as f:
  email_content = f.read()

print(email_content)

Delivered-To: simbajisu@jj.ac.kr
Received: by 2002:a05:7010:45c9:b0:422:302d:1616 with SMTP id x9csp1877715mde;
        Thu, 19 Dec 2024 10:10:00 -0800 (PST)
X-Received: by 2002:a05:6902:cc4:b0:e4b:fe5:5db6 with SMTP id 3f1490d57ef6-e536222f61cmr7367365276.48.1734631800273;
        Thu, 19 Dec 2024 10:10:00 -0800 (PST)
ARC-Seal: i=1; a=rsa-sha256; t=1734631800; cv=none;
        d=google.com; s=arc-20240605;
        b=hs1g8zfS0k9c9l9q6Zqji17lpCyp3ng0UkxHC/qExgUDvAuhg52kF9GvG37fZjx3AM
         9L7J+wyJ7topif9pdpQ8iCrHgIy7KNODNb+aIwPiuZV1rY4n7esj6BRD3DB9B3l3nF1S
         cUfG/Xx0butuhYZWDmnnE4yhtehpBnPcPM/EFmWvL71mqtYbgU+AnSKrdwy+U4VcAmXU
         Ak1z3gg1ofsB2M/RAsDFOYZdjlc/jrQ+5mHZf04apqL5RtP7cA1u2NZa4/Lb/Qe1/jEp
         jY3ymt9xgseq7+2vSciHXVZRj7+3GvNiAsP7TkryPO1tOuB5IQDBUkP2+em4WY40Q039
         hbcQ==
ARC-Message-Signature: i=1; a=rsa-sha256; c=relaxed/relaxed; d=google.com; s=arc-20240605;
        h=to:from:subject:message-id:list-id:feedback-id:precedence
         :list-unsubscrib

In [ ]:
email_conversation = """
From: BTS (bts_0613@naver.com)
To: Jisoo (simbajisu@jj.ac.kr)
Subject: “ZENESIS” bike distribution cooperation and meeting schedule proposal
지수 ,

I am John, Senior Executive Director at Bike Corporation. I recently learned about your new bicycle model, "ZENESIS," through your press release. Bike Corporation is a company that leads innovation and quality in the field of bicycle manufacturing and distribution, with long-time experience and expertise in this field.

We would like to request a detailed brochure for the ZENESIS model. In particular, we need information on technical specifications, battery performance, and design aspects. This information will help us further refine our proposed distribution strategy and marketing plan.

Additionally, to discuss the possibilities for collaboration in more detail, I propose a meeting next Tuesday, January 15th, at 10:00 AM. Would it be possible to meet at your office to have this discussion?

Thank you.

Best regards,
John
Senior Executive Director
Bike Corporation
"""

### Use Output Parser

In [ ]:
class EmailSummary(BaseModel):
  person: str = Field(description="The sender of the email")
  email: str = Field(description="The email adress of the sender")
  subject: str = Field(description="The subject of the email")
  summary: str = Field(description = "A summary of the email content")
  date: str = Field(
      dedcription = "The meeting date and time mentioned in the email content"
  )

In [ ]:
from langchain.output_parsers import PydanticOutputParser

Parser = PydanticOutputParser(pydantic_object = EmailSummary)

In [ ]:
Parser.get_format_instructions()

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"person": {"description": "The sender of the email", "title": "Person", "type": "string"}, "email": {"description": "The email adress of the sender", "title": "Email", "type": "string"}, "subject": {"description": "The subject of the email", "title": "Subject", "type": "string"}, "summary": {"description": "A summary of the email content", "title": "Summary", "type": "string"}, "date": {"dedcription": "The meeting date and time mentioned in the email content", "title": "Date", "type": "string"}}, "required": ["person", 

### Peompt 제작

In [ ]:
prompt = PromptTemplate.from_template(
    """
You are a helpful assistant.

QUESTION:
{question}

EMAIL CONVERSATION:
{email_conversation}

FORMAT:
{format}
"""
)

In [ ]:
prompt = prompt.partial(format = Parser.get_format_instructions())

In [ ]:
prompt

PromptTemplate(input_variables=['email_conversation', 'question'], input_types={}, partial_variables={'format': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"person": {"description": "The sender of the email", "title": "Person", "type": "string"}, "email": {"description": "The email adress of the sender", "title": "Email", "type": "string"}, "subject": {"description": "The subject of the email", "title": "Subject", "type": "string"}, "summary": {"description": "A summary of the email content", "title": "Summary", "type": "string"}, "date": {"dedcription": "The mee

In [ ]:
chain = prompt | llm

In [ ]:
response = chain.stream(
    {
        "email_conversation": email_conversation,
        "question": "이메일을 요약해"
    }
)

In [ ]:
output = stream_response(response, return_output = True)

```json
{
  "person": "John",
  "email": "John@bikecorporation.me",
  "subject": "“ZENESIS” bike distribution cooperation and meeting schedule proposal",
  "summary": "John, Senior Executive Director at Bike Corporation, is interested in the ZENESIS bike model and requests a detailed brochure with technical specifications, battery performance, and design aspects. He proposes a meeting to discuss potential collaboration on Tuesday, January 15th, at 10:00 AM at Kim's office.",
  "date": "Tuesday, January 15th, 10:00 AM"
}
```

# with Parser

In [ ]:
chain = prompt | llm | Parser

In [ ]:
response = chain.invoke(
    {
        "email_conversation": email_conversation,
        "question": "Extract the main content of the email.",
    }
)

# The results are output in the form of an EmailSummary object.
print(response)

person='John' email='John@bikecorporation.me' subject='“ZENESIS” bike distribution cooperation and meeting schedule proposal' summary="John, Senior Executive Director at Bike Corporation, is interested in the ZENESIS bicycle model and requests a detailed brochure with technical specifications, battery performance, and design aspects. He proposes a meeting to discuss collaboration possibilities on Tuesday, January 15th, at 10:00 AM at Kim's office." date='Tuesday, January 15th, at 10:00 AM'


In [ ]:
response.person

'John'